In [4]:
import pandas as pd

In [8]:
df = pd.read_csv('reddit_comments.csv')
df.shape

(10331, 2)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10331 entries, 0 to 10330
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   author   10331 non-null  object
 1   comment  10331 non-null  object
dtypes: object(2)
memory usage: 161.5+ KB


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.model_selection import train_test_split

# Download NLTK tokenizer
nltk.download("punkt")

# Load dataset from CSV
df = pd.read_csv("cleaned_reddit_comments.csv")  

# Function to split comments into sentences
def preprocess_comments(comments):
    sentences = []
    for comment in comments:
        for sentence in sent_tokenize(comment):
            sentences.append(f"<|startoftext|> {sentence} <|endoftext|>") 
    return sentences


# Preprocess the dataset (sentence-level)
sentences = preprocess_comments(df['comment'].dropna().tolist())
sentences = sentences[:1000]
print(sentences[:5])
# Train-validation split (90% train, 10% validation)
train_texts, val_texts = train_test_split(sentences, test_size=0.1, random_state=42)
print("Train Test Split is Completed")
# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Ensure tokenizer has padding token
tokenizer.pad_token = tokenizer.eos_token

# Create dataset dictionary
dataset = DatasetDict({
    "train": Dataset.from_dict({"text": train_texts}),
    "validation": Dataset.from_dict({"text": val_texts}),
})

# Fix: Add labels for loss calculation
def tokenize_function(examples):
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=50)
    tokenized["labels"] = tokenized["input_ids"].copy()  # Add labels
    return tokenized

# Tokenize datasets
tokenized_datasets = dataset.map(tokenize_function, batched=True)

train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["validation"]

# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=1000,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",  
    save_strategy="epoch",      
    learning_rate=0.001,
    lr_scheduler_type="cosine",
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset, 
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)

# Save the model and tokenizer
model.save_pretrained("./gpt2_reddit_model")
tokenizer.save_pretrained("./gpt2_reddit_model")

print("Model training complete and saved!")


/Users/vishnu/Desktop/Text-Completion/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /Users/vishnu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['<|startoftext|> my comment will be buried but what he heck i was probably 8 or 9 at the time, trampolines were the best shit in the world. <|endoftext|>', '<|startoftext|> one day doing flips i lost my sense of reality or something and went head first strait into the ground with my eyes squeezing tight knowing my fate is at hand...i open my eyes and my head is inches above the ground and im dangling there. <|endoftext|>', '<|startoftext|> my legs caught the space between the springs luckily, and i knew i had nothing to do with it. <|endoftext|>', '<|startoftext|> i wasnt trying to save myself but i believe God was watchin out for me. <|endoftext|>', '<|startoftext|> thanks Jesus, I love you reddit dont downvote me lol TLDR flying spaghetti monster lassowed me inches from certain death! <|endoftext|>']
Train Test Split is Completed


Map: 100%|██████████| 100/100 [00:00<00:00, 2816.05 examples/s]
/Users/vishnu/Desktop/Text-Completion/.venv/lib/python3.10/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,1.783200,1.645998
2,1.518900,1.714984
3,1.195200,1.891808


Evaluation Results: {'eval_loss': 1.891808271408081, 'eval_runtime': 2.552, 'eval_samples_per_second': 39.184, 'eval_steps_per_second': 2.743, 'epoch': 3.0}
Model training complete and saved!
